In [1]:
import cv2
import mediapipe as mp
import sys

#미디어파이프 객체 생성
mp_drawing = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh
mp_drawing_styles = mp.solutions.drawing_styles  

# 비디오 캡처 객체 생성
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Camera is not opened")
    sys.exit(1)

# facemesh 객체 생성
face_mesh = mp_face_mesh.FaceMesh()

move_count = 0
last_movement = None
eye_lie = 0 # 눈동자 거짓말 측정

# 마지막 눈동자 움직임 측정(오른쪽, 왼쪽)
last_right_eye_x = 0
last_left_eye_x = 0

# 눈동자 움직임을 감지할 임계값 설정
movement_threshold = 5  # 눈동자가 이 값보다 크게 움직였을 때 움직임으로 간주

while True:
    ret, frame = cap.read()

    if not ret:
        print("Failed to grab frame")
        break

    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # facemesh를 이용하여 얼굴 랜드마크 감지
    results = face_mesh.process(rgb_frame)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            left_eye = face_landmarks.landmark[33]  # 왼쪽 눈의 중심점
            right_eye = face_landmarks.landmark[263]  # 오른쪽 눈의 중심점

            # 눈동자 X 좌표
            right_eye_x = right_eye.x * frame.shape[1]
            left_eye_x = left_eye.x * frame.shape[1]

            # 눈동자 움직임 체크
            if last_right_eye_x != 0 and abs(right_eye_x - last_right_eye_x) > movement_threshold:
                move_count += 1
            # if last_left_eye_x != 0 and abs(left_eye_x - last_left_eye_x) > movement_threshold:
            #     move_count += 1

            # 눈동자 위치 업데이트
            last_right_eye_x = right_eye_x
            last_left_eye_x = left_eye_x

            # 눈동자 이동이 2회 이상 발생하면 동작
            if move_count >= 2:
                # 특정 동작 수행
                print("Eye movement detected!")
                eye_lie += 1
                move_count = 0

            # 랜드마크 그리기
            mp_drawing.draw_landmarks(
                image=frame,
                landmark_list=face_landmarks,
                connections=mp_face_mesh.FACEMESH_TESSELATION,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style(),
            )

    cv2.imshow("Face Mesh", frame)

    key = cv2.waitKey(1)
    if key == 27:
        print("eye_lie : ", eye_lie)
        break

# 종료 시 리소스 해제
cap.release()
cv2.destroyAllWindows()


Eye movement detected!
Eye movement detected!
Eye movement detected!
Eye movement detected!
Eye movement detected!
eye_lie :  5
